In [20]:
import pandas as pd
creds = pd.read_csv("datasets/credits.csv")
print(creds.head())
print(creds.columns)


                                                cast  \
0  [{'cast_id': 14, 'character': 'Woody (voice)',...   
1  [{'cast_id': 1, 'character': 'Alan Parrish', '...   
2  [{'cast_id': 2, 'character': 'Max Goldman', 'c...   
3  [{'cast_id': 1, 'character': "Savannah 'Vannah...   
4  [{'cast_id': 1, 'character': 'George Banks', '...   

                                                crew     id  
0  [{'credit_id': '52fe4284c3a36847f8024f49', 'de...    862  
1  [{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...   8844  
2  [{'credit_id': '52fe466a9251416c75077a89', 'de...  15602  
3  [{'credit_id': '52fe44779251416c91011acb', 'de...  31357  
4  [{'credit_id': '52fe44959251416c75039ed7', 'de...  11862  
Index(['cast', 'crew', 'id'], dtype='object')


In [21]:
movies_meta = pd.read_csv("datasets/movies_metadata.csv")
print(movies_meta.head())
print(movies_meta.columns)
print(movies_meta["production_countries"])
print(movies_meta["release_date"])
movies_meta = movies_meta.fillna("")
print("CLEAN DF")
print(movies_meta.head())

   adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                                NaN  65000000   
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3  False                                                NaN  16000000   
4  False  {'id': 96871, 'name': 'Father of the Bride Col...         0   

                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4                     [{'id': 35, 'name': 'Comedy'}]   

                               homepage     id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story    862  tt0114709                en   
1                                   NaN   8844  tt0113497         

/tmp/ipykernel_5255/1533399231.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_meta = pd.read_csv("datasets/movies_metadata.csv")


In [22]:


import networkx as nx
from networkx.algorithms import bipartite
import ast
graph = nx.Graph()
creds["id"] = creds["id"].astype(str)
work_df = pd.merge(movies_meta[["title", "id", "genres", "original_language", "release_date", "production_countries"]], creds[["cast", "id"]], on= "id", how ="inner")
print(work_df.head())
print(work_df["cast"].iloc[0])
titles = work_df["title"].tolist()
genres = work_df["genres"].tolist()
languages = work_df["original_language"].tolist()
dates = work_df["release_date"].tolist()
countries = work_df["production_countries"].tolist()
cast = work_df["cast"].tolist()
tupled_movie = zip(titles, genres, languages, dates, countries, cast)
edges = []
actors = []
movies = []
for el in list(tupled_movie):
    genres = ast.literal_eval(el[1])
    genres = [genre["name"] for genre in genres]
    if el[4] != "" and isinstance(el[4], str):
        try:
            countries = ast.literal_eval(el[4])
        except (ValueError, SyntaxError):
            countries = []
    else:
        countries = []
    clean_country = []
    if isinstance(countries,list) and len(countries) > 0:  
        for country in countries:
            if country["name"]:
                clean_country.append(country["name"])
    year = el[3]
    if year[:4].isdigit():
        year = int(year[:4])
    else:
        year = ""
    movie_attributes = {"genres": genres, "countries": clean_country, "language": el[2], "release_date": year}
    el_list = ast.literal_eval(el[-1])
    # to avoid disconnected graphs 
    movies.append(("movie: "+ str(el[0]), movie_attributes))
    for actor in el_list:
        gender = "male" if actor["gender"] == 2 else "female"
        edge_attributes = {"character" : actor["character"]}
        if len(clean_country) > 0 and clean_country != None:
            clean_country = clean_country
        else:
            clean_country = []
        actor_attributes = {"gender": gender, "countries" : clean_country, "year":year, "genres":genres}
        # solves self loops for biographies
        actors.append(("actor: "+ actor["name"], actor_attributes))
        edges.append(("movie: "+ str(el[0]), "actor: "+ actor["name"], edge_attributes))


                         title     id  \
0                    Toy Story    862   
1                      Jumanji   8844   
2             Grumpier Old Men  15602   
3            Waiting to Exhale  31357   
4  Father of the Bride Part II  11862   

                                              genres original_language  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...                en   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...                en   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...                en   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...                en   
4                     [{'id': 35, 'name': 'Comedy'}]                en   

  release_date                               production_countries  \
0   1995-10-30  [{'iso_3166_1': 'US', 'name': 'United States o...   
1   1995-12-15  [{'iso_3166_1': 'US', 'name': 'United States o...   
2   1995-12-22  [{'iso_3166_1': 'US', 'name': 'United States o...   
3   1995-12-22  [{'iso_3166_1': 

In [23]:
print(len(movies))
print(len(actors)) 
print(len(edges))
print(actors[0])
# I want there to be a unique node for each actor
clean_actors_dict = {}

for name, attrs in actors:
    if name not in clean_actors_dict:

        new_attrs = attrs.copy()
        new_attrs["countries"] = set(attrs.get("countries") or [])
        genres = attrs.get("genres") or []
        genre_dict = dict()
        for genre in genres:
            genre_dict[genre] = 1
        new_attrs["genres"] = genre_dict
        year = attrs.get("year") or 0
        year_dict = dict()
        year_dict["old_hollywood"] = 0
        year_dict["new_hollywood"] = 0
        if year> 0 and year< 1967:
            year_dict["old_hollywood"] +=1
        elif year >= 1967:
            year_dict["new_hollywood"] +=1
        new_attrs["year"] = year_dict
        clean_actors_dict[name] = new_attrs
    else:   
        extra_countries = set(attrs.get("countries") or [])
        clean_actors_dict[name]["countries"].update(extra_countries)
        extra_genres = attrs.get("genres") or []
        for genre in extra_genres:
            if genre not in clean_actors_dict[name]["genres"]:
                clean_actors_dict[name]["genres"][genre] = 1
            else: 
                clean_actors_dict[name]["genres"][genre] +=1
        current_year = attrs["year"] or 0
        if current_year> 0 and current_year< 1967:
            clean_actors_dict[name]["year"]["old_hollywood"] +=1
        elif current_year >= 1967:
            clean_actors_dict[name]["year"]["new_hollywood"] +=1

# Final pass: Convert sets back to lists and finalize B-prefixed Job IDs
final_actors = []

for name, attrs in clean_actors_dict.items():
    attrs["countries"] = list(attrs["countries"])
    genres = list(attrs["genres"].items())
    sorted_genres = sorted(genres, key=lambda x: x[1], reverse = True)
    if len(sorted_genres)> 0:
        attrs["genre"] = sorted_genres[0][0]
    else:
        attrs["genre"] = ""
    attrs["all_genres"] = [name for name, _ in sorted_genres]
    periods = list(attrs["year"].items())
    del attrs["genres"]
    sorted_periods = sorted(periods, key=lambda x: x[1], reverse = True)
    attrs["period"] = sorted_periods[0][0]
    del attrs["year"]
    final_actors.append((name, attrs))

print(f"Unique actors found: {len(final_actors)}")
print(final_actors[0])



45538
563112
563112
('actor: Tom Hanks', {'gender': 'male', 'countries': ['United States of America'], 'year': 1995, 'genres': ['Animation', 'Comedy', 'Family']})
Unique actors found: 202747
('actor: Tom Hanks', {'gender': 'male', 'countries': ['Mexico', 'Australia', 'Singapore', 'Italy', 'France', 'Malta', 'Hong Kong', 'United Kingdom', 'Germany', 'United States of America', 'Hungary', 'India'], 'genre': 'Comedy', 'all_genres': ['Comedy', 'Drama', 'Documentary', 'Family', 'Animation', 'Thriller', 'Romance', 'History', 'Fantasy', 'Crime', 'Action', 'War', 'Adventure', 'Science Fiction', 'Horror', 'Mystery', 'Foreign', 'TV Movie'], 'period': 'new_hollywood'})


In [24]:
print(final_actors[0])

('actor: Tom Hanks', {'gender': 'male', 'countries': ['Mexico', 'Australia', 'Singapore', 'Italy', 'France', 'Malta', 'Hong Kong', 'United Kingdom', 'Germany', 'United States of America', 'Hungary', 'India'], 'genre': 'Comedy', 'all_genres': ['Comedy', 'Drama', 'Documentary', 'Family', 'Animation', 'Thriller', 'Romance', 'History', 'Fantasy', 'Crime', 'Action', 'War', 'Adventure', 'Science Fiction', 'Horror', 'Mystery', 'Foreign', 'TV Movie'], 'period': 'new_hollywood'})


In [25]:
import networkx as nx 
graph = nx.Graph()
for movie, diz in movies:
    graph.add_node(movie, **diz, bipartite = 0)
for actor, diz in final_actors:
    if not isinstance(diz["genre"], str):
        print(diz)
    graph.add_node(actor, **diz, bipartite = 1)
for edge1, edge2, diz in edges:
    graph.add_edge(edge1, edge2, **diz)

In [26]:
movie_nodes = {n for n, d in graph.nodes(data=True) if d["bipartite"] == 0}
actor_nodes = {n for n, d in graph.nodes(data=True) if d["bipartite"] == 1}

In [27]:
print("GRAPH METRICS\n")
print("Number of nodes:", graph.number_of_nodes())
print("Number of edges:", graph.number_of_edges())
print("Graph density:", nx.bipartite.density(graph, list(actor_nodes)),2)
avg_actors_per_movie = graph.size() / len(movie_nodes)
avg_movies_per_actor = graph.size() / len(actor_nodes)

print(f"Average number of actors per movie: {round(avg_actors_per_movie,2)}")
print(f"Average number of movies per actor: {round(avg_movies_per_actor,2)}")
print("Is there any self loop?", len(list(nx.selfloop_edges(graph))))
if len(list(nx.selfloop_edges(graph))) > 0:
    print(list(nx.selfloop_edges(graph))) # fixed in the previous cells




GRAPH METRICS

Number of nodes: 245024
Number of edges: 560576
Graph density: 6.539972190481372e-05 2
Average number of actors per movie: 13.26
Average number of movies per actor: 2.76
Is there any self loop? 0


In [28]:
movie_nodes = {n for n, d in graph.nodes(data=True) if d["bipartite"] == 0}
actor_nodes = set(graph) - movie_nodes
print("DEGREE CENTRALITY\n")
i = 0
actor_deg = dict(nx.bipartite.degrees(graph, movie_nodes)[0])
movie_deg = dict(nx.bipartite.degrees(graph, actor_nodes)[0])
centr = list(nx.bipartite.degree_centrality(graph, nodes=movie_nodes).items())
centr_actors = [tup for tup in centr if tup[0][:5] == "actor"]
centr_movies = [tup for tup in centr if tup[0][:5] == "movie"]
centr_movies = sorted(centr_movies, key=lambda x: x[1], reverse=True)
centr_actors = sorted(centr_actors, key=lambda x: x[1], reverse=True)
for movie in centr_movies[:10]:
    print(movie[0][6:], "-", "centrality :", round(movie[1], 6), ", cast :", movie_deg[movie[0]], "actors\n")
for actor in centr_actors[:10]:
    print(actor[0][6:], "-", "centrality :", round(actor[1], 6), ", appearances :", actor_deg[actor[0]], "movies\n")

DEGREE CENTRALITY

 Les Misérables - centrality : 0.00215 , cast : 436 actors

 Around the World in Eighty Days - centrality : 0.001539 , cast : 312 actors

 Beauty and the Beast - centrality : 0.001179 , cast : 239 actors

 Alice in Wonderland - centrality : 0.001169 , cast : 237 actors

 Rock of Ages - centrality : 0.001105 , cast : 224 actors

 Mr. Smith Goes to Washington - centrality : 0.001051 , cast : 213 actors

 Jason Bourne - centrality : 0.001026 , cast : 208 actors

 Titanic - centrality : 0.000972 , cast : 197 actors

 Union Pacific - centrality : 0.000908 , cast : 184 actors

 You Don't Mess with the Zohan - centrality : 0.000903 , cast : 183 actors

 Bess Flowers - centrality : 0.005677 , appearances : 240 movies

 Christopher Lee - centrality : 0.003501 , appearances : 148 movies

 John Wayne - centrality : 0.002957 , appearances : 125 movies

 Samuel L. Jackson - centrality : 0.002909 , appearances : 123 movies

 Gérard Depardieu - centrality : 0.002578 , appearances :

In [29]:
print("Is the graph connected?", nx.is_connected(graph))
print("Connected components:", nx.number_connected_components(graph))

Is the graph connected? False
Connected components: 3533


In [30]:
import json
to_dump = nx.readwrite.json_graph.node_link_data(graph)
with open("full_graph.json", "w") as f:
    json.dump(to_dump, f)


In [31]:
# let's create a graph only for the movies produced in Hollywood (movies produced by the US, with co-production with other countries allowed)
hollywood_graph = nx.Graph()

movie_counter = 0
for movie, diz in movies:
    if "United States of America" in diz["countries"]:
        movie_counter += 1
        hollywood_graph.add_node(movie, **diz, bipartite = 0)
print(movie_counter)
actor_counter = 0
for actor, diz in final_actors:
    if "United States of America" in diz["countries"]:
        actor_counter +=1
        hollywood_graph.add_node(actor, **diz, bipartite = 1)
print(actor_counter)
edge_counter = 0
for edge1, edge2, diz in edges:
    if edge1 in hollywood_graph.nodes() and edge2 in hollywood_graph.nodes():
        edge_counter +=1
        hollywood_graph.add_edge(edge1, edge2, **diz)

print(edge_counter)


21182
111458
330284


In [32]:
print("Is the graph connected?", nx.is_connected(hollywood_graph))
print("Connected components:", nx.number_connected_components(hollywood_graph))

Is the graph connected? False
Connected components: 869


In [33]:
movie_nodes = {n for n, d in hollywood_graph.nodes(data=True) if d["bipartite"] == 0}
actor_nodes = set(hollywood_graph) - movie_nodes
print("DEGREE CENTRALITY\n")
i = 0
actor_deg = dict(nx.bipartite.degrees(hollywood_graph, movie_nodes)[0])
movie_deg = dict(nx.bipartite.degrees(hollywood_graph, actor_nodes)[0])
centr = list(nx.bipartite.degree_centrality(hollywood_graph, nodes=movie_nodes).items())
centr_actors = [tup for tup in centr if tup[0][:5] == "actor"]
centr_movies = [tup for tup in centr if tup[0][:5] == "movie"]
centr_movies = sorted(centr_movies, key=lambda x: x[1], reverse=True)
centr_actors = sorted(centr_actors, key=lambda x: x[1], reverse=True)
for movie in centr_movies[:10]:
    print(movie[0][6:], "-", "centrality :", round(movie[1], 6), ", cast :", movie_deg[movie[0]], "actors\n")
for actor in centr_actors[:10]:
    print(actor[0][6:], "-", "centrality :", round(actor[1], 6), ", appearances :", actor_deg[actor[0]], "movies\n")

DEGREE CENTRALITY

 Around the World in Eighty Days - centrality : 0.002799 , cast : 312 actors

 Les Misérables - centrality : 0.002602 , cast : 290 actors

 Rock of Ages - centrality : 0.00201 , cast : 224 actors

 Alice in Wonderland - centrality : 0.00192 , cast : 214 actors

 Mr. Smith Goes to Washington - centrality : 0.001911 , cast : 213 actors

 Jason Bourne - centrality : 0.001866 , cast : 208 actors

 Beauty and the Beast - centrality : 0.00183 , cast : 204 actors

 Titanic - centrality : 0.001767 , cast : 197 actors

 Union Pacific - centrality : 0.001651 , cast : 184 actors

 You Don't Mess with the Zohan - centrality : 0.001642 , cast : 183 actors

 Bess Flowers - centrality : 0.011346 , appearances : 229 movies

 John Wayne - centrality : 0.006044 , appearances : 122 movies

 Samuel L. Jackson - centrality : 0.005747 , appearances : 116 movies

 John Carradine - centrality : 0.005103 , appearances : 103 movies

 Danny Trejo - centrality : 0.004608 , appearances : 93 movi

In [34]:
import json
to_dump = nx.readwrite.json_graph.node_link_data(hollywood_graph)
with open("hollywood_graph.json", "w") as f:
    json.dump(to_dump, f)
